In [16]:
#Use the conda env machine learning to work things
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import scipy.sparse as sparse
import re
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import os.path

In [4]:
motif_df=pd.read_csv('Insulator_motif_dictionary_ext.csv')
motif_df.head()

,Unnamed: 0,CAGGG,AGGGT,GGGTT,GGTTG,GTTGC,TTGCC,TGCCC,GCCCA,CCCAG,...,TTAGN,TAGNN,AGNNN,GNNNN,GGCTN,GCTNN,CTNNN,NNNGG,NNGGC,NGGCC
0,0,14,12,10,4,6,9,12,8,19,...,0,0,0,0,0,0,0,0,0,0
1,1,33,9,5,2,2,5,28,19,44,...,0,0,0,0,0,0,0,0,0,0
2,2,23,6,6,5,4,7,10,13,27,...,0,0,0,0,0,0,0,0,0,0
3,3,7,5,7,7,14,15,11,9,12,...,0,0,0,0,0,0,0,0,0,0
4,4,11,4,8,7,11,9,15,13,27,...,0,0,0,0,0,0,0,0,0,0


In [5]:
random_sequences = pd.read_csv('1000_8kb_random_samples_upper.tab',sep='\t',names=['Region','Sequence'])
insulator_sequences = pd.read_csv('insulator_library.tab',sep='\t',names=['Region','Sequence'])
combined_sequences=insulator_sequences.append(random_sequences)
X_full_data=motif_df
X_full_data.index = range(len(X_full_data))

X_full_data_indexed = X_full_data.iloc[:, 1::]

ins=[1]*len(insulator_sequences)

rando=[0]*len(random_sequences)
data=pd.DataFrame(ins)
scores=data.append(rando)

y_full_data = np.ravel(scores)
X_full_data_indexed.head()

,CAGGG,AGGGT,GGGTT,GGTTG,GTTGC,TTGCC,TGCCC,GCCCA,CCCAG,CCAGT,...,TTAGN,TAGNN,AGNNN,GNNNN,GGCTN,GCTNN,CTNNN,NNNGG,NNGGC,NGGCC
0,14,12,10,4,6,9,12,8,19,12,...,0,0,0,0,0,0,0,0,0,0
1,33,9,5,2,2,5,28,19,44,6,...,0,0,0,0,0,0,0,0,0,0
2,23,6,6,5,4,7,10,13,27,10,...,0,0,0,0,0,0,0,0,0,0
3,7,5,7,7,14,15,11,9,12,7,...,0,0,0,0,0,0,0,0,0,0
4,11,4,8,7,11,9,15,13,27,12,...,0,0,0,0,0,0,0,0,0,0


In [13]:
X_full=X_full_data_indexed.copy()
y_full=y_full_data.copy()
X_train, X_test,y_train,y_test = train_test_split(X_full, y_full, random_state = 1)
y_train_out=pd.DataFrame(y_train)
y_test_out=pd.DataFrame(y_test)

In [15]:
X_train.to_csv('train_data_frame.csv')
y_train_out.to_csv('train_targets.csv')
X_test.to_csv('test_data_frame.csv')
y_test_out.to_csv('test_targets.csv')

In [1]:
def convert(infile,labelfile,outfile,mapper,worddim,batchsize,labelname,dataname,isseq):
    with open(infile) as seqfile, open(labelfile) as labelfile:
        cnt = 0
        seqdata = []
        label = []
        batchnum = 0
        for x,y in izip(seqfile,labelfile):
            if isseq:
                seqdata.append(list(x.strip().split()[1]))
            else:
                seqdata.append(map(float,x.strip().split()))
            #label.append(float(y.strip()))
            label.append(map(float,y.strip().split()))
            cnt = (cnt+1)% batchsize
            if cnt == 0:
                batchnum = batchnum + 1
                seqdata = np.asarray(seqdata)
                label = np.asarray(label)
                t_outfile = outfile + '.batch' + str(batchnum)
                if isseq:
                    seq2feature(seqdata,mapper,label,t_outfile,worddim,labelname,dataname)
                else:
                    feature2feature(seqdata,mapper,label,t_outfile,worddim,labelname,dataname)
                seqdata = []
                label = []
        if cnt >0:
            batchnum = batchnum + 1
            seqdata = np.asarray(seqdata)
            label = np.asarray(label)
            t_outfile = outfile + '.batch' + str(batchnum)
            if isseq:
                seq2feature(seqdata,mapper,label,t_outfile,worddim,labelname,dataname)
            else:
                feature2feature(seqdata,mapper,label,t_outfile,worddim,labelname,dataname)
    return batchnum

def seq2feature(data,mapper,label,out_filename,worddim,labelname,dataname):
    out = []
    for seq in data:
        mat = embed(seq,mapper,worddim)
        result = mat.transpose()
        result1 = [ [a] for a in result]
        out.append(result1)
    outputHDF5(np.asarray(out),label,out_filename,labelname,dataname)

def feature2feature(data,mapper,label,out_filename,worddim,labelname,dataname):
    out = np.asarray(data)[:,None,None,:]
    outputHDF5(out,label,out_filename,labelname,dataname)


def embed(seq,mapper,worddim):
    mat = np.asarray([mapper[element] if element in mapper else np.random.rand(worddim)*2-1 for element in seq])
    return mat

def seq2feature_siamese(data1,data2,mapper,label,out_filename,worddim,labelname,dataname):
    out = []
    datalen = len(data1)
    for dataidx in range(datalen):
        mat = np.asarray([embed(data1[dataidx],mapper,worddim),embed(data2[dataidx],mapper,worddim)])
        result = mat.transpose((2,0,1))
        out.append(result)
    outputHDF5(np.asarray(out),label,out_filename,labelname,dataname)
